#### Creating characters

- brainstorming
- deciding to write something into store inside the 'character object'

##### Loading env vars

In [1]:
import os
from dotenv import load_dotenv

has_anything_loaded = load_dotenv()

if not has_anything_loaded:
    raise ValueError("No .env file found")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

##### Story skeleton as a pydantic model

In [2]:
from pydantic import BaseModel, Field
import uuid

class CharactersRelationTo(BaseModel):
    id: str
    self: str
    family: str
    relatives: str
    friends: str
    coworkers: str
    strangers: str
    authorityFigures: str
    locations: str
    
class CharactersCurrentStateOf(BaseModel):
    id: str
    physical: str
    mental: str
    spiritual: str
    
class CharactersIllness(BaseModel):
    id: str
    physical: str
    mental: str

class Character(BaseModel):
    id: str
    name: str
    age: int
    personality: str
    motivation: str
    pastChildhood: str
    pastTeenage: str
    pastYoungUpUntilNow: str
    strength: str
    weakness: str
    dailyRoutine: str
    phobias: str
    
    relationTo: CharactersRelationTo
    currentStateOf: CharactersCurrentStateOf
    illness: CharactersIllness
    
class Location(BaseModel):
    id: str
    name: str
    description: str
    
class Beat(BaseModel):
    id: str
    name: str
    actions: str = Field(description="The actions or events that take place which move the story forward")
    
class Scene(BaseModel):
    id: str
    name: str
    location: str
    dayOfTime: str
    beats: list[Beat]

class Sequence(BaseModel):
    id: str
    name: str
    scenes: list[Scene]    
    
class Act(BaseModel):
    id: str
    name: str
    summary: str
    sequences: list[Sequence]
    
class Episode(BaseModel):
    id: str
    theme: str
    acts: list[Beat]
    #TODO: fill out the rest of the fields

class Season(BaseModel):
    id: str
    theme: str
    episodes: list[Episode]
    #TODO: fill out the rest of the fields

#TODO: further descriptions and optimizations for the rest of pydantic classes

class Story(BaseModel):
    id: str = Field(default=str(uuid.uuid4()), description="unique identifier for the story")
    genre: str = Field(default="#toBeDefined", description="the genre of the story")
    logline: str = Field(default="#toBeDefined", description="the logline of the story")
    summary: str = Field(default="#toBeDefined", description="the summary of the story")
    plotPoints: list[str] = Field(default=[], description="a list of plot points that describe what happens throughout the entire story")
    character_ids: list[str] = Field(default=[], description="the characters' ids that move the story forward")
    season_ids: list[str] = Field(default=[], description="list of season ids")
    # characters: list[Character] = Field(description="the characters' ids that move the story forward")
    # seasons: list[Season] = Field(description="list of season ids")


##### sample story skeleton as a json string

In [3]:
# ──────────────────────────────────────────────────────────────────────── #
#  Example payload that satisfies *all* Pydantic models with *empty* data
# ──────────────────────────────────────────────────────────────────────── #
story_json = {
    # ─────── Story ────── #
    "id": "123",
    "genre": "",
    "logline": "",
    "summary": "",
    "plotPoints": [],

    # ─────── Characters ────── #
    "characters": [
        {
            "id": "",
            "name": "",
            "age": 0,
            "personality": "",
            "motivation": "",
            "pastChildhood": "",
            "pastTeenage": "",
            "pastYoungUpUntilNow": "",
            "strength": "",
            "weakness": "",
            "dailyRoutine": "",
            "phobias": "",

            "relationTo": {
                "id": "",
                "self": "",
                "family": "",
                "relatives": "",
                "friends": "",
                "coworkers": "",
                "strangers": "",
                "authorityFigures": "",
                "locations": ""
            },

            "currentStateOf": {
                "id": "",
                "physical": "",
                "mental": "",
                "spiritual": ""
            },

            "illness": {
                "id": "",
                "physical": "",
                "mental": ""
            }
        }
    ],

    # # ─────── Locations (referenced by scenes) ────── #
    # "locations": [
    #     {"id": "", "name": "", "description": ""}
    # ],

    # ─────── Seasons ────── #
    "seasons": [
        {
            "id": "",
            "theme": "",
            "episodes": [
                {
                    "id": "",
                    "theme": "",

                    # Episode → acts (type `list[Act]`)
                    "acts": [
                        {
                            "id": "123",
                            "name": "",
                            "summary": "",

                            # Act → sequences (empty list – “empty act”)
                            "sequences": [
                                # One empty Sequence (no scenes → “empty sequence”)
                                {
                                    "id": "",
                                    "name": "",
                                    "scenes": [
                                        # One empty Scene (no beats → “empty scene”)
                                        {
                                            "id": "",
                                            "name": "",
                                            "location": "",
                                            "dayOfTime": "",
                                            # Beat list left empty (no actions)
                                            "beats": [
                                                {
                                                    "id": "",
                                                    "name": "",
                                                    "actions": ""
                                                }
                                            ]
                                        }
                                    ]
                                }
                            ]
                        }
                    ]
                }
            ]
        }
    ]
}
# ──────────────────────────────────────────────────────────────────────── #

# ──────────────────────────────────────────────────────────────────────── #
#  How to validate it against your models
# ──────────────────────────────────────────────────────────────────────── #
# from your_models import Story        # ← the file where you defined the classes
# story = Story.parse_obj(story_json)   # → no validation errors


# story = Story.model_validate(story_json, by_alias=True)
# story = Story(**story_json)
# story_json

##### Tools for persistance

In [4]:
from typing import Any
from langchain.tools import tool, ToolRuntime
from pydantic import ValidationError
from typing import Union

stories_in_memory_db: dict[str, Story] = {}

@tool
def read_or_create_story(story_id: Union[str, None], runtime: ToolRuntime) -> str:
    """
    Searches for a story with the given story_id. If no such story is found, it creates a new story and returns it.
    
    Args:
        story_id: the id of the story that wants to be read. This is optional. You can pass 'None' and it will create a new story.
    
    Return: story object
    """
    writer = runtime.stream_writer
    writer("read story is called")
    
    def create_new_story():
        writer("creating a new story")
        new_story = Story()
        stories_in_memory_db[new_story.id] = new_story
        return new_story.__str__()
        
    if story_id is None:
        return create_new_story()    
    
    writer("checking the in-memory db for the story with id {}".format(story_id))
    result = stories_in_memory_db.get(story_id)
    
    if result is None:
        return create_new_story()
    else:
        return result.__str__()

@tool
def update_story(updated_story: dict, runtime: ToolRuntime) -> str:
    """
    Overwrites the existing outdated story with the given updated_story
    
    Args:
        updated_story: contains the new version of story as a json. it should contain all the fields of a Story object.
    """
    
    writer = runtime.stream_writer
    
    writer("update_story is called")
    print("update_story is called")
    try:
        print("validation begins")
        validated_story = Story.model_validate(updated_story)
        print("validation succeeded")
        stories_in_memory_db[validated_story.id] = validated_story
        print("updated the story in-memory")
        print("returning")
        return "story successfully updated. the new story: {}".format(validated_story.__str__())
    
    except ValidationError as ve:
        print("Validation of the story str failed")
        print(ve.errors())
        print("returning")
        return "updating story failed"
    


##### Agent definition

In [5]:
from langchain_ollama.chat_models import ChatOllama
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver

gpt_llm = ChatOllama(
    model="gpt-oss:20b", 
    reasoning="high"
)

my_agent = create_agent(
    gpt_llm,
    tools=[read_or_create_story, update_story],
    checkpointer=InMemorySaver(), # this retains the conversation history
    system_prompt="""
            You are an expert screenwriter. You are here to provide ideas and inspirations.
            Do not update something before I confirm it.
            Do not make up ids.
            Tell what you did, even if you only updated something.
            Always go through the story json fields one by one and do not try to answer more than one field at a time. 
            Provide new ideas and critics to the user's ideas and decisions. Base those ideas and critics on past scenarios, movies and even maybe other art forms.
            Do not sugercoat anything or do not try to make it look pleasing.
            
            """,
    response_format=ToolStrategy(Story) #TODO: this might get in the way of generating other structured data. Other than Story.
)


##### Running the agent

In [15]:
bool()

False

In [18]:
from langchain.messages import AIMessage, AIMessageChunk
from IPython.display import Markdown, display
from typing import Union, Literal

len_messages_before_model_call = 1
most_recent_message_type = AIMessageChunk
type_of_content: Union[Literal['reasoning'], Literal['text'], None]

while True:
    
    user_input = input("You: ")
    if user_input.lower() in {"exit", "quit"}:
        break
    
    print("user: {}".format(user_input))
    
    # response = my_agent.invoke(
    #         {'messages': [{"role": "user", "content": user_input}]},
    #         {'configurable': {"thread_id": 1}},
    #     )
    
    # latest_messages = response['messages'][len_messages_before_model_call:]
    # for latest_message in latest_messages:
    #     latest_message_content = latest_message.content
    #     display(Markdown(latest_message_content))
    
    # len_messages_before_model_call = len(response['messages'])
    # ---
    for chunk in my_agent.stream(
        {'messages': [{'role': 'user', 'content': user_input}]}, 
        {'configurable': {'thread_id': 1}}, 
        # stream_mode='updates'
        stream_mode='messages'
        ):
        # print('chunk {}'.format(chunk))
        current_message = chunk[0] # type: ignore
        # print('first message: {}'.format(current_message))
        
        
        if type(current_message) != most_recent_message_type:
            print('old: {}, new: {}'.format(most_recent_message_type, type(current_message)))
            print("") # leave an empty line so that it looks clean when a different message type is being printed
            
        # Determine the type of current response whether it is a reasoning chunk or a chunk of an actual response.
        if len(current_message.content) > 0:
            type_of_content = 'text'
        elif len(current_message.content) == 0 and bool(current_message.additional_kwargs.get('reasoning_content', '')):
            type_of_content = 'reasoning'
        else:
            type_of_content = 'text'
            
        # Depending on the content type, get the relevant field and print it out
        if type_of_content == 'text':
            content_blocks_of_current_message = current_message.content_blocks
            if len(content_blocks_of_current_message) > 0 and 'text' in content_blocks_of_current_message[0]:
                current_text = content_blocks_of_current_message[0].get('text', '')
                print(current_text, end="", flush=True) 
        
        elif type_of_content == 'reasoning':
            reasoning_of_current_message = current_message.additional_kwargs.get('reasoning_content', '')
            if len(reasoning_of_current_message) > 0:
                print(reasoning_of_current_message, end="", flush=True)
               
        
        else:
            print("the type is not AIMessage or AIMessageChunk, but {}".format(type(current_message)))            
        


user: i want to write a story
The user repeatedly says "i want to write a story" and does not answer the genre question. According to instructions: "Always go through the story json fields one by one and do not try to answer more than one field at a time." Also: "Do not update something before I confirm it." "Tell what you did, even if you only updated something."

We have a story object with fields: character_ids, genre, logline, summary, plotPoints, season_ids. We have to proceed field by field. We already created a new story with id and left genre as '#toBeDefined'. Next, we need to get genre. The user keeps saying "i want to write a story" but not specifying genre. We can ask for genre. But instructions say we should not ask for more than one field at a time, but we can ask. But maybe we can interpret: "Let’s go through fields one by one." So we should ask for genre, then for logline, summary, etc.

We should respond politely, maybe ask for the genre. The user is not responding. Pe